In [1]:
import requests
import pandas as pd
from glom import glom
from ast import literal_eval
import json

In [2]:
url_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=56"
resposta = requests.get(url_deputados).json()

deputados_dataframe = pd.DataFrame(resposta['dados']).drop(columns=["uri", "uriPartido", "urlFoto", "email"])

projetos = ["PL 2337/21", "PLP 19/19", "PL 591/2021", "PL 3729/04", "PL 3723/19", "PEC 06/2019", "PEC 125/2011", "MP 1045/21", "MP 1031/21", "PL 10372/18", "PEC 135/19", "PEC 241/2016"]

dicionario = {
  "2288389-78": "PL 2337/21",
  "2265124-70": "PLP 19/19",
  "2270894-103": "PL 591/2021",
  "257161-337": "PL 3729/04", 
  "2209381-100": "PL 3723/19", 
  "2192459-786": "PEC 06/2019", 
  "531331-321": "PEC 125/2011", 
  "2279513-64": "MP 1045/21",
  "2270789-157": "MP 1031/21", 
  "2178170-53": "PL 10372/18", 
  "2220292-229": "PEC 135/19", 
  "2088351-324": "PEC 241/2016"
}

loc = 5

for projeto in projetos:
    deputados_dataframe.insert(loc, column=projeto, value="")
    loc += 1
    
deputados_dataframe.to_csv("deputados.csv", index=False)

In [23]:
ids_votacoes = ["2288389-78", "2265124-70", "2270894-103", "257161-337", "2209381-100", "2192459-786", "531331-321", "2279513-64", "2270789-157", "2178170-53", "2220292-229", "2088351-324"]

for id in ids_votacoes:
    url_votacoes = "https://dadosabertos.camara.leg.br/api/v2/votacoes/" + id + "/votos"

    resposta = requests.get(url_votacoes).json()
    
    votacoes_dataframe = pd.DataFrame(resposta['dados'])
    
    print("Iniciando " + str(id) + "...")
    
    for index, row in votacoes_dataframe.iterrows():    
        if row["tipoVoto"] == "Sim":
            deputados_dataframe.loc[deputados_dataframe["id"] == row["deputado_"]["id"], [dicionario[id]]] = 1
        elif row["tipoVoto"] == "Não":
            deputados_dataframe.loc[deputados_dataframe["id"] == row["deputado_"]["id"], [dicionario[id]]] = 0
        else:
            deputados_dataframe.loc[deputados_dataframe["id"] == row["deputado_"]["id"], [dicionario[id]]] = 0.5
        
deputados_dataframe.replace("", 0.5, inplace=True)
deputados_dataframe.to_csv("deputados.csv", index=False)

Iniciando 2288389-78...
Iniciando 2265124-70...
Iniciando 2270894-103...
Iniciando 257161-337...
Iniciando 2209381-100...
Iniciando 2192459-786...
Iniciando 531331-321...
Iniciando 2279513-64...
Iniciando 2270789-157...
Iniciando 2178170-53...
Iniciando 2220292-229...
Iniciando 2088351-324...
